In [ ]:
import os, json, joblib, dgl, torch
import numpy as np
from feature_store import fetch_edge_features
from eval_utils import make_eval_loader
from eval_infer import infer_split
from eval_metrics import compute_metrics
from eval_roc import plot_roc_ovr, false_negatives_by_class

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load artifacts
g_val = dgl.load_graphs("graphs/val.bin")[0][0]
fs_val = "feature_store/val"

# label map
with open("artifacts/label_map.json","r",encoding="utf-8") as f:
    label2id = json.load(f)
id2label = {v:k for k,v in label2id.items()}
classes  = [id2label[i] for i in range(len(label2id))]

# Rebuild the model (must match your training config)
edge_in = fetch_edge_features(np.load(os.path.join(fs_val,"edge_indices.npy"))[:1], fs_val).shape[1]
hidden  = 128  # set to what you trained with
num_classes = len(label2id)

from train_edgecls_dbg import _FallbackEdgeGraphSAGE  # your class
model = _FallbackEdgeGraphSAGE(node_in=0, edge_in=edge_in, hidden=hidden, num_classes=num_classes, dropout=0.3).to(device)
model.load_state_dict(torch.load("artifacts/best_edge_sage.pt", map_location=device))
model.eval()

# Build loader for VAL
val_loader = make_eval_loader(g_val, fs_val, fanouts=(25,15), batch_size=2048)

# Inference
y_true, y_pred, y_prob = infer_split(model, val_loader, g_val, device, fetch_edge_features)

# Metrics
metrics = compute_metrics(y_true, y_pred, label2id)
print(json.dumps({
    "accuracy": metrics["accuracy"],
    "macro": metrics["macro"],
    "macro_FAR": metrics["macro_FAR"]
}, indent=2))
# Confusion Matrix & FN
cm = np.array(metrics["confusion_matrix"])
fn_by_class = false_negatives_by_class(cm, classes)
print("False negatives by class:", fn_by_class)

# Save metrics
os.makedirs("artifacts", exist_ok=True)
with open("artifacts/metrics_val.json","w") as f:
    json.dump(metrics, f, indent=2)

# ROC curves
plot_roc_ovr(y_true, y_prob, classes, out_path="artifacts/roc_val.png")


ModuleNotFoundError: No module named 'model'